Desarrollo de EndPoints

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Abrimos y leemos el dataset unficado
ruta_unificado = "..\\Datasets\\unificado.parquet"
df_unificado = pd.read_parquet(ruta_unificado)

In [7]:
# Observemos las Filas, Columnas y tipo de datos
print(f'Filas : {df_unificado.shape[0]}, Variables : {df_unificado.shape[1]}')
df_unificado.info()

Filas : 8183355, Variables : 12
<class 'pandas.core.frame.DataFrame'>
Index: 8183355 entries, 0 to 8344483
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   genres              object 
 1   title               object 
 2   year                object 
 3   price               float64
 4   item_id             int32  
 5   developer           object 
 6   user_id             object 
 7   recommend           bool   
 8   sentiment_analysis  int64  
 9   items_count         int64  
 10  playtime_forever    float64
 11  playtime_2weeks     float64
dtypes: bool(1), float64(3), int32(1), int64(2), object(5)
memory usage: 725.8+ MB


Se crearan conjuntos de dataset para dar respuestas a los distintos endpoints

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [11]:
df_userdata = df_unificado[['price','user_id','recommend','items_count']]               
df_userdata.to_parquet("../Datasets/userdata.parquet")
df_userdata.shape

(8183355, 4)

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [15]:
df_userforgenre = df_unificado[['user_id', 'playtime_forever', 'genres', 'year']]
df_userforgenre.to_parquet("../Datasets/userforgenre.parquet")
df_userforgenre.shape

(8183355, 4)

**Endpoint 4**. Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos) <br>
**Endpoint 5**. Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [18]:
df_bdev_devre = df_unificado[['developer', 'recommend', 'year', 'sentiment_analysis']]
df_bdev_devre.to_parquet('../Datasets/endpoint_4_5.parquet')
df_bdev_devre.shape

(8183355, 4)

In [19]:
df_recomendacion = df_unificado[['item_id', 'title', 'genres', 'playtime_2weeks', 'recommend', 'sentiment_analysis', ]]
df_recomendacion.head(2)

,item_id,title,genres,playtime_2weeks,recommend,sentiment_analysis
0,282010,Carmageddon Max Pack,Action,0.0,True,1
1,282010,Carmageddon Max Pack,Action,0.0,True,1


In [20]:
# Filtrar por Recomnedado
df_recomendacion = df_recomendacion[df_recomendacion['recommend']==True]

# Filtrar por analisis de sentimiento igual a 2 (Positivo)
df_recomendacion = df_recomendacion[df_recomendacion['sentiment_analysis'] == 2]

# Borrar variable recommend
df_recomendacion = df_recomendacion.drop(['recommend'], axis=1)

In [ ]:
"""Sumamos la cantidad de horas jugadas por juego en las últimas dos semanas y la cantidad de recoendaciones positivas, agrupado por juego. Tomamos la cantidad jugada en las últimas dos semanas y lo usamos para el modelo de recomendación, entendiendo que esta cantidad de horas da una tendencia para poder recomendar. No tenemos demasiados parámetros para comparar similitud de coseno, por lo que una tendencia puede ser, en algunos casos, una buena manera de recomendar."""
df_recomendacion = df_recomendacion.groupby(['item_id', 'title', 'genres'], as_index=False).agg({
    'sentiment_analysis': 'count',
    'playtime_2weeks': 'sum',
 })

# Ordenamos de manera descendente en función del análisis de sentimiento.
df_recomendacion = df_recomendacion.sort_values(by='sentiment_analysis', ascending=False)
df_recomendacion = df_recomendacion.groupby(['item_id', 'title']).agg({
    'genres': lambda x: list(x),
    'sentiment_analysis': 'sum',
    'playtime_2weeks': 'sum'
}).reset_index()